## LSTM 셀 구현
지금까지 RNN을 살펴보았다면 이제 LSTM 셀과 LSTM 계층을 살펴보겠습니다. 이번 예제에서 사용할 데이터셋은 MNIST입니다.

MNIST는 손으로 쓴 숫자 이미지(0~9까지 값을 갖는 고정 크기 이미지 (28px x 28px))들로 구성되어 있습니다.

먼저 필요한 라이브러리들을 호출합니다.

In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dataset

from torch.autograd import Variable
from torch.nn import Parameter # 파라미터 목록을 가지고 있는 라이브러리
from torch import Tensor
from torch.utils.data import DataLoader

import torch.nn.functional as F

import math

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

torch.manual_seed(125)

print(device)


c:\Users\aqs45\OneDrive\바탕 화면\repo\Learn_pytorch\venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda
